[View in Colaboratory](https://colab.research.google.com/github/GeniGaus/MLBlr/blob/master/DNSTUpdated.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 40
num_filter = 16
compression = 0.8
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 7s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
BatchNormInit = BatchNormalization()(input)
reluInit = Activation('relu')(BatchNormInit)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(reluInit)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 3)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
43776/50000 [=========================>....] - ETA: 32s - loss: 1.6756 - acc: 0.3824

50000/50000 [==============================] - 273s 5ms/step - loss: 1.6426 - acc: 0.3951 - val_loss: 1.4867 - val_acc: 0.4808
Epoch 2/50
26240/50000 [==============>...............] - ETA: 1:55 - loss: 1.3148 - acc: 0.5228

50000/50000 [==============================] - 258s 5ms/step - loss: 1.2518 - acc: 0.5462 - val_loss: 1.2115 - val_acc: 0.5891
Epoch 3/50
19584/50000 [==========>...................] - ETA: 2:27 - loss: 1.0761 - acc: 0.6119

50000/50000 [==============================] - 258s 5ms/step - loss: 1.0371 - acc: 0.6280 - val_loss: 1.2632 - val_acc: 0.5984
Epoch 4/50
17024/50000 [=========>....................] - ETA: 2:39 - loss: 0.9238 - acc: 0.6696

50000/50000 [==============================] - 258s 5ms/step - loss: 0.9033 - acc: 0.6785 - val_loss: 1.3959 - val_acc: 0.5895
Epoch 5/50
16064/50000 [========>.....................] - ETA: 2:44 - loss: 0.8228 - acc: 0.7087

...

50000/50000 [==============================] - 257s 5ms/step - loss: 0.2274 - acc: 0.9200 - val_loss: 0.4742 - val_acc: 0.8671
Epoch 47/50
15360/50000 [========>.....................] - ETA: 2:47 - loss: 0.2143 - acc: 0.9250

50000/50000 [==============================] - 257s 5ms/step - loss: 0.2190 - acc: 0.9217 - val_loss: 0.5782 - val_acc: 0.8534
Epoch 48/50
15360/50000 [========>.....................] - ETA: 2:47 - loss: 0.2108 - acc: 0.9242

50000/50000 [==============================] - 257s 5ms/step - loss: 0.2214 - acc: 0.9204 - val_loss: 0.5336 - val_acc: 0.8580
Epoch 49/50
15360/50000 [========>.....................] - ETA: 2:44 - loss: 0.2119 - acc: 0.9242

50000/50000 [==============================] - 256s 5ms/step - loss: 0.2178 - acc: 0.9227 - val_loss: 0.6948 - val_acc: 0.8288
Epoch 50/50
15360/50000 [========>.....................] - ETA: 2:46 - loss: 0.2074 - acc: 0.9233

50000/50000 [==============================] - 257s 5ms/step - loss: 0.2118 - acc: 0.9235 - val_loss: 0.6026 - val_acc: 0.8445


In [17]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 0.6026396932601928
Test accuracy: 0.8445


In [15]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_5_bs64_dropout02_cmp08_filters16.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [16]:
# Saving using PyDrive to google drive

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'DNST_model_5_bs64_dropout02_cmp08_filters16.h5'})
uploaded.SetContentFile('DNST_model_5_bs64_dropout02_cmp08_filters16.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14NZE7globLk7EftpXP3eKkA_DFekTt1H
